# Database interface

**The database interface is old and not recommend for new users, due to it being very specialized.**  It does, however, provide a way to operate on a lot of data without loading it all into memory.  The module remains since it provides the backend to `niimpy.read_sqlite`.

We currently do not document the database interface well, and this is legacy documentation.  If this proves useful to someone, we will go and improve.

Most data in databases would be read via `read.read_sqlite` instead:
```python
df = niimpy.read_sqlite(FILE_NAME, TABLE_NAME, tz=TZ)
```



## Initial setup

In [1]:
import niimpy
data = niimpy.open(niimpy.sampledata.DATA, tz='Europe/Helsinki')

## Conventions

### Common column names
* `time`: unixtime, integer or float
* `ts`: pandas.Timestamp
* DataFrame and Series indexes, wherever possible, are returned as pandas.DateTimeIndexes (which is a pandas.Timestamp)
*

## Common arguments

* `table=`: First required positional argument.

* `user=`: Second required positional argument. User ID (str) to use to filter data.  This is always a required argument, but there are two special values:
  * Use `niimpy.ALL` for all users.
  * Use `None` for single-user databases.
  
* `start=`, `end=`: Limit the range of selected data.  The times can be given in different formats: unixtime (int or float), string (parsed smartly with dateutil, this seems to be localtime), or a Python `datetime.datetime` object (python seems to interpert naive datetime objects as localtime).

* `limit=<int>`: Return at most this many results.  This can be useful for initial testing: select a few tens or hundreds of results to see if it works before selecting everything.

* `offset=<int>`: Companion of `limit`: how many values to skip when returning results. 

## Multi-user databases

There is a distinction between a single-user database and multi-user database: the difference is that a multi-user database has a `user` and `device` columns, a single-user database doesn't.  (The single-user database wolud be better called "single-device").  This distinction is sort of historical, but serves as a somewhat useful way to distinguish between the "easy" case, and cases where you *have* to consider multiple devices.

All functions take a `user` argument, which selects the user to return within the data.  They also, in the future, take a `device` argument which filters on device name.

The `user` argument must also be given for single-user databases, but should be given as `niimpy.ALL`.

## Database functions: metadata
These functions give you information about the data within the database

### Users in database: `Database.users`
Returns `None` if single-user database otherwise a `set` of usernames.

In [2]:
data.users()

### Tables in database: `Database.tables`
Returns all tables in database

In [3]:
data.tables()

{'AwareScreen'}

### Amount of data per user and per table: `Database.user_table_counts`
Returns a `pandas.DataFrame` with rows of different tables (converters) and columns of users.  The values are count of data for that (converter, user).

The single-user version has a more limited form, rows are tables and there is one column, `count`.

In [4]:
data.user_table_counts()

count
AwareScreen  1156

### First, last timestamp in database: `Database.first`, `Database.last`
Let's say you want to find the first and last datapoint for a certain (converter, user).  Remember, for single-user databases we use `None` as the user argument.

Output is a trivial DataFrame with columns `time` (unixtime) and `datetime` (pandas.Timestamp).

In [5]:
data.first('AwareScreen', None)

time                            datetime
0  1.531171e+09 2018-07-10 00:13:17.933000192+03:00

In [6]:
data.first('AwareScreen', None)['datetime'][0]

Timestamp('2018-07-10 00:13:17.933000192+0300', tz='Europe/Helsinki')

In [7]:
data.first('AwareScreen', None)['datetime'][0].strftime('%Y-%m-%d')

'2018-07-10'

### Count of data for (converter, user): `Database.count`
Basically the same as first/last timestamp:

In [8]:
data.count('AwareScreen', None)

count
0   1156

In [9]:
data.count('AwareScreen', None)['count'][0]

1156

## Accessing data: database methods
These functions all access data from a database `data` and return data frames.

### Raw data: `Database.raw`
This returns the raw data in a table.  It can tell you the columns, etc.

In [10]:
data.raw("AwareScreen", None).head(5)

time screen_status   
2018-07-10 00:13:17.933000192+03:00  1.531171e+09             1  \
2018-07-10 00:13:28.672000+03:00     1.531171e+09             0   
2018-07-10 00:13:28.717999872+03:00  1.531171e+09             2   
2018-07-10 12:05:05.299000064+03:00  1.531214e+09             1   
2018-07-10 12:05:09.783000064+03:00  1.531214e+09             0   

                                                               datetime  
2018-07-10 00:13:17.933000192+03:00 2018-07-10 00:13:17.933000192+03:00  
2018-07-10 00:13:28.672000+03:00       2018-07-10 00:13:28.672000+03:00  
2018-07-10 00:13:28.717999872+03:00 2018-07-10 00:13:28.717999872+03:00  
2018-07-10 12:05:05.299000064+03:00 2018-07-10 12:05:05.299000064+03:00  
2018-07-10 12:05:09.783000064+03:00 2018-07-10 12:05:09.783000064+03:00

### Timestamps of data: `Database.timestamps`
This returns the timestamps of all data, but not the data itself (this assumes that the data is created with a `time` column that has unixtime in it - which is the Koota standard).

There are two different ways data can be returned:
* If the database is multi-user and a `user` argument is **not** given, return a DataFrame with index of the timestamps and a `user` column.
* If the database is multi-user and a `user` argument is given, return only a `pandas.Series` with that user's and table's timestamps.
* If the database is single-user, only return a `pandas.Series` with the timestamps (there can be no confusion).

In [11]:
data.timestamps("AwareScreen", None).head()

0   2018-07-10 00:13:17.933000192+03:00
1      2018-07-10 00:13:28.672000+03:00
2   2018-07-10 00:13:28.717999872+03:00
3   2018-07-10 12:05:05.299000064+03:00
4   2018-07-10 12:05:09.783000064+03:00
Name: time, dtype: datetime64[ns, Europe/Helsinki]

### Data hourly summaries: `Database.hourly`

In [12]:
data.hourly("AwareScreen", None, columns=['screen_status']).head(1)

day  hour  count  screen_status_mean   
2018-07-10 00:00:00+03:00  2018-07-10     0      3                 1.0  \

                          screen_status_std  screen_status_count  
2018-07-10 00:00:00+03:00              None                    3

If you give it a list of columns, it will give you the mean/standard deviation/count 

In [13]:
data.hourly("AwareScreen", None, columns=['screen_status']).head(1)

day  hour  count  screen_status_mean   
2018-07-10 00:00:00+03:00  2018-07-10     0      3                 1.0  \

                          screen_status_std  screen_status_count  
2018-07-10 00:00:00+03:00              None                    3

### Data occurrence in intervals: `Database.occurrence`

*Note: this function calculates occurrence from the database.  There is also a utility function which calculates occurance from time serieses (described below).  They should have the same effect, but the other function is more flexible.*

This makes a measure of data occurrence for sensors which should be continually sending data.  To do this, it:
* Divides all time into hours
* Divides all hours into five 12-minute intervals
* Count the number of 12-minute intervals that have data.  This is $occurrence$
* For each hour, report $occurrence$.  If it is 5, then assume we have data which is somewhat uniformly occuring in time.  If it is 0, then we had no data.

This isn't the perfect measure, but is reasonably effective and simple to calculate.  For data which isn't continuous (like screen data we are actually using), it shows how much the sensor has been used.

Column meanings: `day` is obvious, `hour` is hour of day, `occurrence` is the measure described above, `count` is total number of data points in this hour, `withdata` is which of the 12-min intervals (0-4) have data.

In [14]:
data.occurrence("AwareScreen", None).head()

day  hour  occurrence  count withdata
2018-07-10 00:00:00+03:00  2018-07-10     0           1      3        1
2018-07-10 12:00:00+03:00  2018-07-10    12           4     18  0,1,2,3
2018-07-10 14:00:00+03:00  2018-07-10    14           2      6      1,3
2018-07-10 15:00:00+03:00  2018-07-10    15           3     13    2,3,4
2018-07-10 19:00:00+03:00  2018-07-10    19           2      7      0,3

## Miscelaneous calculations on databases

### Sum of survey scores: `Database.get_survey_score`

TODO: needs further documenting and an example.

The `get_survey_score` is a convenience method to get the sum of scores of a survey.  It can only be used on Survey tables.

It has the standard `table` and `user` arguments, a `survey` argument (filters for survey questions, this is a prefix for the "id" column).

TODO: get sample data and use it.

In [15]:
#data.get_survey_score(table='HyksSurveyAllAnswers', user=niimpy.ALL, survey='PHQ9')